In [1]:
import neurokit2 as nk
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as matplotlib
import numpy as np
import os

from neurokit2.misc import NeuroKitWarning
from neurokit2.signal.signal_rate import _signal_rate_plot
from neurokit2.ecg.ecg_peaks import _ecg_peaks_plot
from neurokit2.ecg.ecg_segment import ecg_segment

# Paths to folders
raw_data_folder = '/Users/firefly/Desktop/Team_Messung/data/rawdata'
results_folder = '/Users/firefly/Desktop/Team_Messung/data/results'
os.makedirs(results_folder, exist_ok=True)

# Parameters
participants = ['sub-01', 'sub-02']
conditions1 = ['empty', 'full']
conditions2 = ['bright', 'dark']

all_results = []

# Process each participant and condition
for pi in participants:
    for c1 in conditions1:
        for c2 in conditions2:
            # Assemble file name
            filename = f"{raw_data_folder}/{pi}_{c1}-{c2}_ecg.csv"
            print(f'\nProcessing: {filename}')
            
            try:
                # Read ECG data
                subdata = pd.read_csv(filename, header=None, names=['ECG'], skiprows=1)
                ecg_data = subdata['ECG'].values
                
                print(f"Data length: {len(ecg_data)} samples")
                
                # Process the full time window
                signals_full, info = nk.ecg_process(ecg_data, sampling_rate=1000)
                
                # Prepare figure with GridSpec
                gs = matplotlib.gridspec.GridSpec(2, 2, width_ratios=[2/3, 1/3])
                fig = plt.figure(figsize=(15, 10))
                
                # Create subplots
                ax0 = fig.add_subplot(gs[0, :-1])
                ax1 = fig.add_subplot(gs[1, :-1], sharex=ax0)
                ax2 = fig.add_subplot(gs[:, -1])
                
                # Plot ECG peaks
                phase = None
                if "ECG_Phase_Ventricular" in signals_full.columns:
                    phase = signals_full["ECG_Phase_Ventricular"].values
                
                ax0 = _ecg_peaks_plot(
                    signals_full["ECG_Clean"].values,
                    info=info,
                    sampling_rate=info["sampling_rate"],
                    raw=signals_full["ECG_Raw"].values,
                    quality=signals_full["ECG_Quality"].values,
                    phase=phase,
                    ax=ax0
                )
                
                # Plot Heart Rate
                ax1 = _signal_rate_plot(
                    signals_full["ECG_Rate"].values,
                    info["ECG_R_Peaks"],
                    sampling_rate=info["sampling_rate"],
                    title="Heart Rate",
                    ytitle="Beats per minute (bpm)",
                    color="#FF5722",
                    color_mean="#FF9800",
                    color_points="#FFC107",
                    ax=ax1
                )
                
                # Plot individual heart beats
                ax2 = ecg_segment(
                    signals_full,
                    info["ECG_R_Peaks"],
                    info["sampling_rate"],
                    show="return",
                    ax=ax2
                )
                
                # Adjust subplot positions
                ax0.set_position([0.1, 0.6, 0.6, 0.35])
                ax1.set_position([0.1, 0.1, 0.6, 0.35])
                ax2.set_position([0.75, 0.1, 0.2, 0.85])
                
                # Set title for the whole figure
                plt.suptitle(f"ECG Analysis - {pi} ({c1}_{c2})", y=0.95)
                
                # Save figure
                figure_filename = f"{results_folder}/{pi}_{c1}_{c2}_ecg_nk.png"
                plt.savefig(figure_filename, bbox_inches='tight', dpi=300)
                print(f"Saved figure to {figure_filename}")
                plt.close()
                
                # Process interval-related features
                print("\nCalculating interval-related features...")
                results = nk.ecg_intervalrelated(signals_full, sampling_rate=1000)
                
                # Add metadata
                results['Participant'] = pi
                results['Condition'] = f"{c1}_{c2}"
                all_results.append(results)
                
                print("Interval-related features:")
                print(results)
                
            except FileNotFoundError:
                print(f'File not found: {filename}')
                continue
            except Exception as e:
                print(f'Error processing {filename}: {str(e)}')
                continue

# Combine all results
print("\nCombining all results...")
final_results = pd.concat(all_results, ignore_index=True)

# Save results
output_filename = f"{results_folder}/ecg_results.csv"
final_results.to_csv(output_filename, index=False)
print(f"\nSaved results to {output_filename}")


Processing: /Users/firefly/Desktop/Team_Messung/data/rawdata/sub-01_empty-bright_ecg.csv
Data length: 299000 samples
Saved figure to /Users/firefly/Desktop/Team_Messung/data/results/sub-01_empty_bright_ecg_nk.png

Calculating interval-related features...
Interval-related features:
  ECG_Rate_Mean            HRV_MeanNN               HRV_SDNN  \
0     72.077112  [[832.586592178771]]  [[72.76489934047386]]   

              HRV_SDANN1             HRV_SDNNI1 HRV_SDANN2 HRV_SDNNI2  \
0  [[49.90068820868521]]  [[56.47501911305945]]    [[nan]]    [[nan]]   

  HRV_SDANN5 HRV_SDNNI5              HRV_RMSSD  ...             HRV_FuzzyEn  \
0    [[nan]]    [[nan]]  [[37.61056529552221]]  ...  [[0.7553832745422128]]   

                 HRV_MSEn               HRV_CMSEn              HRV_RCMSEn  \
0  [[1.3310616958569885]]  [[1.2050325689522794]]  [[1.7892360401848137]]   

                   HRV_CD                HRV_HFD                 HRV_KFD  \
0  [[1.4693234488236355]]  [[1.549911812727842]]  [